In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

In [4]:
#list of experiments on our mlflow ui
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [12]:
#check for the given experiment, what are the best runs/models

from mlflow.entities import ViewType

runs  = client.search_runs(
    experiment_ids="1",
    filter_string ="metrics.rmse <6.8",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ['metrics.rmse ASC']
)

In [13]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse:{run.data.metrics['rmse']:.4f}")

run id: b737d95fc8a14ae2a599771751638cb0, rmse:6.2887
run id: e389dae9d8a045538a7330c618cb7c96, rmse:6.2902
run id: 4baf368bdf8e4633bdd0d3e7dc2c3560, rmse:6.2913
run id: 7338b6a818bf4893b4ccc0887aab1148, rmse:6.3040
run id: 80fe68adcdce401699fd14e4a906c162, rmse:6.3093


In [14]:
#promote some of the above models to model registry
import mlflow

#need to set also here the tracking server otherwise it would try to look for the models in 
#the local folder
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
#register a new version of the model
run_id = '80fe68adcdce401699fd14e4a906c162'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/07/28 16:16:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1659005196460, current_stage='None', description=None, last_updated_timestamp=1659005196460, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [17]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1659001275504, description='The NYC Taxi predictor for trip duration', last_updated_timestamp=1659005196460, latest_versions=[<ModelVersion: creation_timestamp=1659001302451, current_stage='Staging', description='', last_updated_timestamp=1659001594486, name='nyc-taxi-regressor', run_id='08062686756249d8ba1a9b5920d2f1a2', run_link='', source='./mlruns/1/08062686756249d8ba1a9b5920d2f1a2/artifacts/model', status='READY', status_message=None, tags={'model': 'Gradient boosting regressor'}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1659005196460, current_stage='None', description=None, last_updated_timestamp=1659005196460, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [23]:
#get latest version of the model

model_name='nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name = model_name)

In [24]:
for version in latest_versions:
    print(f'version: {version.version}, stage: {version.current_stage}')

version: 2, stage: Staging
version: 3, stage: None


In [29]:
#transition a model from one stage to another
model_version =3
new_stage = 'Staging'

client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage=new_stage,
    archive_existing_versions = False
)

<ModelVersion: creation_timestamp=1659005196460, current_stage='Staging', description=None, last_updated_timestamp=1659006144939, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [32]:
#we can update a version also
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name= model_name,
    version=3,
    description = f'The model version {model_version} was transitioned to {new_stage} on {date}')

<ModelVersion: creation_timestamp=1659005196460, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2022-07-28', last_updated_timestamp=1659006281139, name='nyc-taxi-regressor', run_id='80fe68adcdce401699fd14e4a906c162', run_link=None, source='./mlruns/1/80fe68adcdce401699fd14e4a906c162/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>